# Pre Processing Notebook
* plan to preprocess data for label encoding even
though the data is not ordinal; because I plan to use this for random forest regresoor and
another tree algorithm
* will also think about encoding in one-hot encoding and then perform knn

In [29]:
# import packages

import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import klib
import plotly.figure_factory as ff
from IPython.core.display import display
from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import joblib
# pd.set_option('display.max_columns', None)
# pd.reset_option('max_rows')
#np.set_printoptions(threshold=sys.maxsize)

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set(style='ticks', context='talk')

In [30]:
# load dataset

df_path = r'../../data/processed/mushroom_imputed_2_adj.csv'
mushroom = pd.read_csv(df_path)

mushroom

,Class,Cap-Shape,Cap-Surface,Cap-Color,Bruises,Odor,Gill-Attachment,Gill-Spacing,Gill-Size,Gill-Color,...,Stalk-Surface-Above-Ring,Stalk-Surface-Below-Ring,Stalk-Color-Above-Ring,Stalk-Color-Below-Ring,Veil-Color,Ring-Number,Ring-Type,Spore-Print-Color,Population,Habitat
0,poisonous,convex,smooth,brown,bruises,pungent,free,close,narrow,black,...,smooth,smooth,white,white,white,one,pendant,black,scattered,urban
1,edible,convex,smooth,yellow,bruises,almond,free,close,broad,black,...,smooth,smooth,white,white,white,one,pendant,brown,numerous,grasses
2,edible,bell,smooth,white,bruises,anise,free,close,broad,brown,...,smooth,smooth,white,white,white,one,pendant,brown,numerous,meadows
3,poisonous,convex,scaly,white,bruises,pungent,free,close,narrow,brown,...,smooth,smooth,white,white,white,one,pendant,black,scattered,urban
4,edible,convex,smooth,gray,no,none,free,crowded,broad,black,...,smooth,smooth,white,white,white,one,evanescent,brown,abundant,grasses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8107,edible,convex,smooth,brown,no,none,attached,close,broad,yellow,...,smooth,smooth,orange,orange,brown,one,pendant,buff,several,leaves
8108,edible,flat,smooth,brown,no,none,attached,close,broad,brown,...,smooth,smooth,orange,orange,orange,one,pendant,buff,clustered,leaves
8109,poisonous,knobbed,scaly,brown,no,fishy,free,close,narrow,buff,...,smooth,silky,white,white,white,one,evanescent,white,several,leaves
8110,edible,convex,smooth,brown,no,none,attached,close,broad,yellow,...,smooth,smooth,orange,orange,orange,one,pendant,orange,clustered,leaves


In [31]:
mushroom.describe()

,Class,Cap-Shape,Cap-Surface,Cap-Color,Bruises,Odor,Gill-Attachment,Gill-Spacing,Gill-Size,Gill-Color,...,Stalk-Surface-Above-Ring,Stalk-Surface-Below-Ring,Stalk-Color-Above-Ring,Stalk-Color-Below-Ring,Veil-Color,Ring-Number,Ring-Type,Spore-Print-Color,Population,Habitat
count,8112,8112,8112,8112,8112,8112,8112,8112,8112,8112,...,8112,8112,8112,8112,8112,8112,8112,8112,8112,8112
unique,2,6,4,10,2,9,2,2,2,12,...,4,4,9,9,4,3,5,9,6,7
top,edible,convex,scaly,brown,no,none,free,close,broad,buff,...,smooth,smooth,white,white,white,one,pendant,white,several,woods
freq,4196,3650,3244,2282,4736,3516,7904,6810,5600,1728,...,5169,4929,4454,4374,7914,7486,3956,2378,4040,3148


In [32]:
# our target is class so drop it in predictor variables

X = mushroom.drop('Class', axis=1)
y = mushroom[['Class']]

# Use to check that target variable is not in our predictor set and check predictor set is right
'Class' not in X.columns, 'Class' in y.columns

(True, True)

In [39]:
# split our data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train

,Cap-Shape,Cap-Surface,Cap-Color,Bruises,Odor,Gill-Attachment,Gill-Spacing,Gill-Size,Gill-Color,Stalk-Shape,...,Stalk-Surface-Above-Ring,Stalk-Surface-Below-Ring,Stalk-Color-Above-Ring,Stalk-Color-Below-Ring,Veil-Color,Ring-Number,Ring-Type,Spore-Print-Color,Population,Habitat
7337,knobbed,scaly,red,no,fishy,free,close,narrow,buff,tapering,...,silky,silky,pink,white,white,one,evanescent,white,several,leaves
8104,knobbed,smooth,red,no,fishy,free,close,narrow,buff,tapering,...,silky,smooth,pink,white,white,one,evanescent,white,several,woods
3123,convex,scaly,red,bruises,none,free,close,broad,brown,tapering,...,smooth,smooth,white,gray,white,one,pendant,black,several,woods
7196,knobbed,scaly,red,no,fishy,free,close,narrow,buff,tapering,...,smooth,smooth,pink,white,white,one,evanescent,white,several,woods
2996,convex,scaly,brown,bruises,none,free,close,broad,brown,tapering,...,smooth,smooth,gray,gray,white,one,pendant,black,solitary,woods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226,convex,smooth,gray,bruises,foul,free,close,broad,chocolate,tapering,...,fibrous,smooth,white,white,white,one,pendant,chocolate,scattered,urban
5390,flat,smooth,gray,bruises,foul,free,close,broad,pink,tapering,...,smooth,smooth,white,white,white,one,pendant,chocolate,several,urban
860,flat,scaly,brown,bruises,anise,free,close,broad,white,enlarging,...,smooth,scaly,white,white,white,one,pendant,brown,solitary,paths
7603,bell,smooth,gray,no,none,free,crowded,broad,gray,enlarging,...,smooth,smooth,white,white,white,two,pendant,white,scattered,grasses


We are using a ordinal encoder even though the categories are not ordinal.
This is due to the fact that we will be using a tree model and tree models have a behavior
to perform well with ordinal encoders even when no ordering relationship is present
Also save time in not needing to perform one hot encoding and then pca

In [40]:
# we preprocess the training the data

ordinal_X = OrdinalEncoder()
X_train = ordinal_X.fit_transform(X_train)

ordinal_y = OrdinalEncoder()
y_train = ordinal_y.fit_transform((y_train))

display('X train', X_train, 'y train', y_train)

'X train'

array([[4., 2., 7., ..., 7., 4., 1.],
       [4., 3., 7., ..., 7., 4., 6.],
       [2., 2., 7., ..., 0., 4., 6.],
       ...,
       [3., 2., 0., ..., 1., 5., 3.],
       [0., 3., 3., ..., 7., 3., 0.],
       [4., 0., 3., ..., 7., 2., 0.]])

'y train'

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [35]:
X_test

,Cap-Shape,Cap-Surface,Cap-Color,Bruises,Odor,Gill-Attachment,Gill-Spacing,Gill-Size,Gill-Color,Stalk-Shape,...,Stalk-Surface-Above-Ring,Stalk-Surface-Below-Ring,Stalk-Color-Above-Ring,Stalk-Color-Below-Ring,Veil-Color,Ring-Number,Ring-Type,Spore-Print-Color,Population,Habitat
2157,flat,fibrous,brown,bruises,none,free,close,broad,purple,tapering,...,smooth,smooth,white,gray,white,one,pendant,brown,several,woods
453,convex,fibrous,yellow,bruises,anise,free,crowded,narrow,pink,tapering,...,smooth,smooth,white,white,white,one,pendant,brown,several,woods
8043,convex,smooth,brown,no,none,attached,close,broad,brown,enlarging,...,smooth,smooth,orange,orange,brown,one,pendant,brown,several,leaves
3985,convex,fibrous,gray,no,foul,free,close,broad,pink,enlarging,...,silky,silky,pink,brown,white,one,large,chocolate,solitary,grasses
5366,flat,scaly,yellow,no,foul,free,close,broad,pink,enlarging,...,silky,silky,pink,buff,white,one,large,chocolate,several,woods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6004,convex,scaly,white,no,none,free,close,narrow,white,enlarging,...,smooth,fibrous,white,white,white,one,flaring,chocolate,several,woods
7941,knobbed,scaly,brown,no,foul,free,close,narrow,buff,tapering,...,silky,smooth,pink,white,white,one,evanescent,white,several,paths
5624,flat,scaly,cinnamon,no,musty,attached,close,broad,white,enlarging,...,silky,scaly,cinnamon,cinnamon,white,none,none,white,clustered,woods
7991,bell,smooth,brown,no,none,attached,close,broad,brown,enlarging,...,smooth,smooth,orange,orange,orange,one,pendant,brown,clustered,leaves


In [36]:
# preprocess the test data

X_test = ordinal_X.transform(X_test)

y_test = ordinal_y.transform(y_test)

display('X test', X_test, 'y test', y_test)

'X test'

array([[3., 0., 0., ..., 1., 4., 6.],
       [2., 0., 9., ..., 1., 4., 6.],
       [2., 3., 0., ..., 1., 4., 1.],
       ...,
       [3., 2., 2., ..., 7., 1., 6.],
       [0., 3., 0., ..., 1., 1., 1.],
       [2., 2., 3., ..., 0., 5., 6.]])

'y test'

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]])

In [37]:
# save these preprocessed values for usage on modeling

# savez saves these as compressed files
np.savez('../../data/processed/train.npz', X_train, y_train)
np.savez('../../data/processed/test.npz', X_test, y_test)

In [38]:
# Save our transformers

joblib.dump(ordinal_X, '../../data/processed/ordinal_X')
joblib.dump(ordinal_y, '../../data/processed/ordinal_y')

['../../data/processed/ordinal_y']